# Luminosity Analysis

In this notebook we analyze stellar luminosity within the SMC Wing for both resolved and unresolved stars. In order to determine the total observed stellar luminosity for our region, we completed an IRAF Apphot analysis surrounding the stellar sample (white contour). We further completed seven circular Apphot analyses for the seven most prominent under-resolved star clusters in the region (red circles). Blue points indicate the stars that were resolved by Daophot among these star clusters. The image on which these analyses were applied was a composite GALEX image, "ngc602.fits". The photometered regions are outlined in tetris_clusters_composite.reg. 

<img src="images/clusters_composite_stars.png" style="width: 400px;"/>

## Import modules

In [1]:
# Standard
import numpy as np
import matplotlib.pyplot as plt

## Foreground-corrected photometry

In [2]:
# Original files
data = np.genfromtxt('../data/foreground_corrected_photometry', dtype = None, names = True)
clusters = np.genfromtxt('../data/underresolved_clusters', dtype = None, names = True)

id_number, ra, dec, nuv_v, nuv, b_v, v = data['id'], data['ra'], \
data['dec'], data['nuv_v'], data['nuv'], data['b_v'], data['v'], 

print("There are {} field stars in our sample".format(len(nuv)))

There are 1104 field stars in our sample


## Individual stellar luminosity from observed NUV magnitude

Individual stellar luminosities were calculated as a direct conversion from the DAOPHOT photometry analysis in AB mags to counts / sec.

In [3]:
# Convert NUVo absolute magnitudes to counts per second

# Convert from galactic extinction-corrected absolute AB magnitudes to 
# uncorrected apparent AB magnitudes
    # 18.70 for conversion from absolute to apparent
    # 0.42 for galatic extinction
nuv_apparent = nuv + 18.70 + 0.42

# Convert from AB magnitudes to counts per second (CPS) for stellar sample
# The value 20.08 comes from this document https://asd.gsfc.nasa.gov/archive/galex/FAQ/counts_background.html
counts_resolved = np.sum(10**( (nuv_apparent - 20.08) / -2.5) )
print ("The resolved stellar population has a luminosity of {} counts/sec"\
       .format(np.round(counts_resolved, 1)))

The resolved stellar population has a luminosity of 15150.0 counts/sec


## Entire region luminosity analysis (white contour)
The entire region luminosity was calculated through a IRAF Apphot polygonal analysis

In [4]:
# Define data for the entire region
tetris = clusters[7]

area_tetris = tetris['area']
counts_tetris = tetris['sum']
mean_intensity_tetris = counts_tetris / area_tetris

print('The entire region (white contour) has a luminosity of {} counts/sec over an \
area of {} arcsecond**2, leading to a mean intensity of {} counts/sec/arcsecond**2'\
      .format(counts_tetris, area_tetris, np.round(mean_intensity_tetris, 5)))

The entire region (white contour) has a luminosity of 27341.832 counts/sec over an area of 2139950.4 arcsecond**2, leading to a mean intensity of 0.01278 counts/sec/arcsecond**2


### Background estimate

In [5]:
mean_intensity_background = 0.00305
mean_intensity_background_error = 0.0005

print('The mean intensity of the background is {} +/- {} counts/sec/arcsecond**2, as determined by \
analyzing regions to the east and south of our studied region'.format(mean_intensity_background, \
                                                                      mean_intensity_background_error))

The mean intensity of the background is 0.00305 +/- 0.0005 counts/sec/arcsecond**2, as determined by analyzing regions to the east and south of our studied region


### Background-corrected luminosity

In [6]:
mean_intensity_tetris_corrected = mean_intensity_tetris - mean_intensity_background
counts_tetris_corrected = mean_intensity_tetris_corrected * area_tetris

print('The corrected mean intensity for this region is {} counts/sec/arcsecond**2, \
leading to a corrected luminosity of {} counts/sec for the entire region'\
      .format(np.round(mean_intensity_tetris_corrected, 6), np.round(counts_tetris_corrected, 2)))

The corrected mean intensity for this region is 0.009727 counts/sec/arcsecond**2, leading to a corrected luminosity of 20814.98 counts/sec for the entire region


### Comparison to resolved stellar luminosity analysis

In [7]:
luminosity_percentage_resolved = counts_resolved / counts_tetris_corrected * 100

print('From the above analysis, we find a luminosity of {} counts/sec for resolved stars in this region \
and {} counts/sec for the entire region (including both resolved and unresolved stars, as well as \
potential background error). This analysis tells us that roughly {}% of the luminosity in this region \
can be recovered in our resolved stellar population.'.format(np.round(counts_resolved, 2), \
                                            np.round(counts_tetris_corrected, 2), \
                                            np.round(luminosity_percentage_resolved, 2)))

From the above analysis, we find a luminosity of 15150.02 counts/sec for resolved stars in this region and 20814.98 counts/sec for the entire region (including both resolved and unresolved stars, as well as potential background error). This analysis tells us that roughly 72.78% of the luminosity in this region can be recovered in our resolved stellar population.


## Unresolved cluster stellar luminosity (red circles, blue points)
The unresolved cluster luminosity was calculated through seven IRAF Apphot circular analyses

In [8]:
# Cluster luminosities
cluster = clusters[0:7]

area_clusters = np.sum(cluster['area'])
counts_clusters = np.sum(cluster['sum'])
mean_intensity_clusters = counts_clusters / area_clusters

mean_intensity_clusters_corrected = mean_intensity_clusters - mean_intensity_background
counts_clusters_corrected = mean_intensity_clusters_corrected * area_clusters

luminosity_percentage_clusters = counts_clusters_corrected / counts_tetris_corrected * 100

In [9]:
# "Composite" luminosities, resolved stars in clusters
composite = np.genfromtxt('../data/composite_stars_coordinates', dtype = None, names = True)

ra_composite = composite['ra']
dec_composite = composite['dec']

indices_composite = np.nonzero(np.in1d(ra, ra_composite))[0]
indices_not_composite = [x for x in range(len(ra)) if x not in indices_composite]

nuv_composite = nuv_apparent[indices_composite]
nuv_not_composite = nuv_apparent[indices_not_composite]

counts_composite = np.sum(nuv_composite)
mean_intensity_composite = counts_composite / area_clusters

counts_composite_corrected = mean_intensity_composite * area_clusters
counts_clusters_unresolved_corrected = counts_clusters_corrected - counts_composite_corrected

luminosity_percentage_clusters_unresolved = \
counts_clusters_unresolved_corrected / counts_tetris_corrected * 100

In [10]:
print('After summing the luminosity for the seven clusters analyzed with Apphot, we find the stellar \
luminosity in clusters to be {} counts/sec, which accounts for {}% of the luminosity for the entire \
region. However, these regions also include the few stars that were resolved among the clusters \
(blue points). After subtracting these stars, we get a final luminosity of {} \
for the unresolved stars in clusters, accounting for {}% of the luminosity for \
the entire region.'.format(np.round(counts_clusters_corrected, 2), \
                           np.round(luminosity_percentage_clusters, 2), \
                           np.round(counts_clusters_unresolved_corrected, 2), \
                           np.round(luminosity_percentage_clusters_unresolved, 2)))

After summing the luminosity for the seven clusters analyzed with Apphot, we find the stellar luminosity in clusters to be 9460.07 counts/sec, which accounts for 45.45% of the luminosity for the entire region. However, these regions also include the few stars that were resolved among the clusters (blue points). After subtracting these stars, we get a final luminosity of 5992.05 for the unresolved stars in clusters, accounting for 28.79% of the luminosity for the entire region.


In [11]:
luminosity_percentage_sum = luminosity_percentage_resolved + luminosity_percentage_clusters_unresolved

print('Through independent analyses, we find that {}% of the luminosity in our region \
is recovered in our resolved stellar population, and {}% of the luminosity is recovered \
through the unresolved stars in clusters. This sums to a total luminosity percentage of {}%, \
which is well within our margin for error.'.format(np.round(luminosity_percentage_resolved, 2), \
                                                   np.round(luminosity_percentage_clusters_unresolved, 2), \
                                                  np.round(luminosity_percentage_sum, 2)))

Through independent analyses, we find that 72.78% of the luminosity in our region is recovered in our resolved stellar population, and 28.79% of the luminosity is recovered through the unresolved stars in clusters. This sums to a total luminosity percentage of 101.57%, which is well within our margin for error.
